In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from pathlib import Path

from collections import defaultdict, Counter
from datetime import datetime
import pandas as pd

import requests

from src.etherscan_data import EtherscanData
from src.transactions import TransactionExplorer

In [3]:
ETHERSCAN_API_KEY = "PUT_YOUR_KEY_HERE"

address = "0x7d6149ad9a573a6e2ca6ebf7d4897c1b766841b4"

# Addresses "db"

In [4]:
# We found these addresses to be interesting

addresses_db = {}
addresses_db["0x7d6149ad9a573a6e2ca6ebf7d4897c1b766841b4"] = "ADDRESS_OF_INTEREST"
addresses_db["0x1c11ba15939e1c16ec7ca1678df6160ea2063bc5"] = "Related Party"
addresses_db["0x4678f0a6958e4d2bc4f1baf7bc52e8f3564f3fe4"] = "Maker: Proxy Registry"
addresses_db["0xd848f54280f8fe8661b796e3bb8d8922c87af452"] = "DSProxy"
addresses_db["0x6b175474e89094c44da98b954eedeac495271d0f"] = "DAI Stablecoin"
addresses_db["0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"] = "Wrapped Ether"
addresses_db["0x39755357759ce0d7f32dc8dc45414cca409ae24e"] = "Eth2Dai: Old Contract"
addresses_db["0x0000000000000000000000000000000000000000"] = "ZERO Address"
addresses_db["0x2f0b23f53734252bda2277357e97e1517d6b042a"] = "Maker: MCD Join ETH A"
addresses_db["0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"] = "USD Coin"
addresses_db["0xc73e0383f3aff3215e6f04b0331d58cecf0ab849"] = "Maker Migration"
addresses_db["0xad37fd42185ba63009177058208dd1be4b136e6b"] = "Maker: MCD Join SAI" # 243 tx - не зватает трансфера
addresses_db["0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359"] = "SAI Stablecoin"
addresses_db["0x794e6e91555438afc3ccf1c5076a74f42133d08d"] = "OasisDEX"
addresses_db["0x868ba9aeaca5b73c7c27f3b01588bf4f1339f2bc"] = "VoteProxyFactory"
addresses_db["0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2"] = "Maker: MKR Token"
addresses_db["0xcfeed3fbefe9eb09b37539eaa0ddd58d1e1044ca"] = "VoteProxy" # Создан в транзакции 826 с VoteProxyFactory
addresses_db["0xf9be8f0945acddeedaa64dfca5fe9629d0cf8e5d"] = "PollingEmitter"
addresses_db["0xe41d2489571d322189246dafa5ebde1f4699f498"] = "0x: ZRX Token"
addresses_db["0x5d3a536e4d6dbd6114cc1ead35777bab948e3643"] = "Compound: cDAI Token"
addresses_db["0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407"] = "Compound: cZRX Token"
addresses_db["0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b"] = "Compound: Comptroller"
addresses_db["0xc00e94cb662c3520282e6f5717214004a7f26888"] = "Compound: COMP Token"
addresses_db["0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5"] = "Compound: cETH Token"
addresses_db["0xdac17f958d2ee523a2206206994597c13d831ec7"] = "Tether: USDT Stablecoin"
addresses_db["0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9"] = "Compound: cUSDT Token"
addresses_db["0x960b236a07cf122663c4303350609a66a7b288c0"] = "Aragon: Old ANT Token"
addresses_db["0x408e41876cccdc0f92210600ef50372656052a38"] = "Ren: REN Token"
addresses_db["0xd26114cd6ee289accf82350c8d8487fedb8a0c07"] = "OMG Network: OMG Token"
addresses_db["0xef1f00b67ccd8f3604a20b7a4425ab667f1bd7f9"] = "Related Party 2"
addresses_db["0xc6f7f4c321038e332ff9074038e9defd10a5960f"] = "Related Party 3"
addresses_db["0xd3a9fe267852281a1e6307a1c37cdfd76d39b133"] = "PollingEmitter_2"
addresses_db["0xa74476443119a942de498590fe1f2454d7d4ac0d"] = "Golem Token"
addresses_db["0x221657776846890989a759ba2973e427dff5c9bb"] = "Augur: REPv2 Token"
addresses_db["0x0a3f6849f78076aefadf113f5bed87720274ddc0"] = "Maker: Governance Contract"
addresses_db["0x7a250d5630b4cf539739df2c5dacb4c659f2488d"] = "Uniswap V2: Router 2"
addresses_db["0xf04a5cc80b1e94c69b48f5ee68a08cd2f09a7c3e"] = "GemJoin"
addresses_db["0x1985365e9f78359a9b6ad760e32412f4a445e862"] = "Augur: REP Token"
addresses_db["0xa618e54de493ec29432ebd2ca7f14efbf6ac17f7"] = "IOU Token"
addresses_db["0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7"] = "Curve.fi: DAI/USDC/USDT Pool"
addresses_db["0x39aa39c021dfbae8fac545936693ac917d5e7563"] = "Compound: cUSDC Token"
addresses_db["0x45f783cce6b7ff23b2ab2d70e416cdb7d6055f51"] = "Curve.fi: y Swap"
addresses_db["0x2dded6da1bf5dbdf597c45fcfaa3194e53ecfeaf"] = "Curve.fi: cyDAI/cyUSDT/cyUSDC Pool"
addresses_db["0xe592427a0aece92de3edee1f18e0157c05861564"] = "Uniswap V3: Router"
addresses_db["0x0000000000085d4780b73119b644ae5ecd22b376"] = "TrueUSD: TUSD Token"
addresses_db["0x12392f67bdf24fae0af363c24ac620a2f67dad86"] = "Compound: cTUSD Token"
addresses_db["0x7d2768de32b0b80b7a3454c06bdac94a69ddc7a9"] = "Aave: Lending Pool V2"
addresses_db["0xbcca60bb61934080951369a648fb03df4f96263c"] = "Aave: aUSDC Token V2"
addresses_db["0xd784927ff2f95ba542bfc824c8a8a98f3495f6b5"] = "Aave: Incentives Controller"
addresses_db["0x4da27a545c0c5b758a6ba100e3a049001de870f5"] = "Aave: Staked Aave"
addresses_db["0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9"] = "Aave: AAVE Token"
addresses_db["0xecd5e75afb02efa118af914515d6521aabd189f1"] = "Curve.fi Factory USD Metapool (TUSD3CRV-f)"
addresses_db["0x890f4e345b1daed0367a877a1612f86a1f86985f"] = "Curve.fi: UST MetaPool"
addresses_db["0xa47c8bf37f92abed4a126bda807a7b7498661acd"] = "Terra: USTC Token"
addresses_db["0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45"] = "Uniswap V3: Router 2"
addresses_db["0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852"] = "Uniswap V2: USDT"
addresses_db["0x3416cf6c708da44db2624d63ea0aaef7113527c6"] = "Uniswap V3: USDC-USDT 4"
addresses_db["0x11b815efb8f581194ae79006d24e0d814b7697f6"] = "Uniswap V3: USDT 3"
addresses_db["0x1111111254fb6c44bac0bed2854e76f90643097d"] = "1inch v4: Aggregation Router"
addresses_db["0x55b916ce078ea594c10a874ba67ecc3d62e29822"] = "Unknown contract"
addresses_db["0x2260fac5e5542a773aa44fbcfedf7c193bc2c599"] = "Wrapped BTC: WBTC Token"
addresses_db["0x1111111254eeb25477b68fb85ed929f73a960582"] = "1inch v5: Aggregation Router"
addresses_db["0xae7ab96520de3a18e5e111b5eaab095312d7fe84"] = "Lido: stETH Token"
addresses_db["0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0"] = "Lido: wstETH Token"
addresses_db["0x99a58482bd75cbab83b27ec03ca68ff489b5788f"] = "Curve.fi: Swap Router"
addresses_db["0xa6d19194da3b357a122fc7e35f7ce9ec00dbb496"] = "Unknown contract 1"
addresses_db["0xdd3f50f8a6cafbe9b31a427582963f465e745af8"] = "Rocket Pool: Deposit v1.2"

name_to_address = {}
for k, v in addresses_db.items():
    name_to_address[v] = k

def get_address_name(hash: str):
    return addresses_db.get(hash, hash)

## Get etherscan data

In [5]:
address_etherscan_data = EtherscanData(address, ETHERSCAN_API_KEY)

Found in cache: {'module': 'account', 'action': 'txlist', 'address': '0x7d6149ad9a573a6e2ca6ebf7d4897c1b766841b4', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'txlistinternal', 'address': '0x7d6149ad9a573a6e2ca6ebf7d4897c1b766841b4', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'tokentx', 'address': '0x7d6149ad9a573a6e2ca6ebf7d4897c1b766841b4', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'tokennfttx', 'address': '0x7d6149ad9a573a6e2ca6ebf7d4897c1b766841b4', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'token1155tx', 'address': '0x7d6149ad9a573a6e2ca6ebf7d4897c1b766841b4',

In [6]:
data = address_etherscan_data.get_summary()
data.head(20)

,hash,timestamp,datetime,date,normal,has_internal,erc20,erc721,erc1155
0,0x1956d7a97e04141c75af0900e30e9fa0a813081141fb...,1574146181,2019-11-19 10:49:41,2019-11-19,1,0,0,0,0
1,0x63128cd3e4d74c6f08db5687a49863eb2aa83d2416f6...,1574146262,2019-11-19 10:51:02,2019-11-19,1,0,0,0,0
2,0x8cbd80175791f801a14076683f12c4cc9f2eef790a14...,1574146780,2019-11-19 10:59:40,2019-11-19,1,0,0,0,0
3,0x4d4c3827b2d91039f326ec6205a7f17067e378bb3319...,1574147079,2019-11-19 11:04:39,2019-11-19,1,0,1,0,0
4,0x60ab9b59a03330fbfc7ab0725d719a38f3014e580280...,1574147260,2019-11-19 11:07:40,2019-11-19,1,0,0,0,0
5,0x31d07d90af71a8434025a4336f0bf63c84ef363f6d6f...,1574147280,2019-11-19 11:08:00,2019-11-19,1,0,0,0,0
6,0x0234537bdb61976e26d7d6308699778a908a28ba799a...,1574147343,2019-11-19 11:09:03,2019-11-19,1,0,0,0,0
7,0x9db1a290efcd4ec554c733a4b2708dea32f61c3c9d4b...,1574147391,2019-11-19 11:09:51,2019-11-19,1,0,2,0,0
8,0xfc94e60af55bac23a0cfd992bfea765328cc4f00e573...,1574148663,2019-11-19 11:31:03,2019-11-19,1,0,0,0,0
9,0x39ac821552373eed76807a4025ad7ce5406ef8861a3f...,1574148791,2019-11-19 11:33:11,2019-11-19,1,0,0,0,0


In [7]:
ds_proxy_data = EtherscanData(name_to_address["DSProxy"], ETHERSCAN_API_KEY)

Found in cache: {'module': 'account', 'action': 'txlist', 'address': '0xd848f54280f8fe8661b796e3bb8d8922c87af452', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'txlistinternal', 'address': '0xd848f54280f8fe8661b796e3bb8d8922c87af452', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'tokentx', 'address': '0xd848f54280f8fe8661b796e3bb8d8922c87af452', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'tokennfttx', 'address': '0xd848f54280f8fe8661b796e3bb8d8922c87af452', 'startblock': 0, 'endblock': 99999999, 'sort': 'asc', 'apikey': 'VDXQK313Q3FRMFRV12C47QVCF3XJBRXYNX'}
Found in cache: {'module': 'account', 'action': 'token1155tx', 'address': '0xd848f54280f8fe8661b796e3bb8d8922c87af452',

In [8]:
ds_proxy_summary = ds_proxy_data.get_summary()
ds_proxy_summary.head(20)

,hash,timestamp,datetime,date,normal,has_internal,erc20,erc721,erc1155
0,0x8cbd80175791f801a14076683f12c4cc9f2eef790a14...,1574146780,2019-11-19 10:59:40,2019-11-19,0,1,0,0,0
1,0x4d4c3827b2d91039f326ec6205a7f17067e378bb3319...,1574147079,2019-11-19 11:04:39,2019-11-19,1,1,1,0,0
2,0x39ac821552373eed76807a4025ad7ce5406ef8861a3f...,1574148791,2019-11-19 11:33:11,2019-11-19,1,1,1,0,0
3,0x8027f5a6fb1260d8f19f869ac9ec973784e1f5bcdeff...,1574149718,2019-11-19 11:48:38,2019-11-19,1,0,0,0,0
4,0x7a9b6e4529bba4dd3d4339516d3668ab1b3ff955c87f...,1574156204,2019-11-19 13:36:44,2019-11-19,1,1,1,0,0
5,0x3eb4cb71a9cf4352c5509a7e1bc4c2bc9b7cc9e8473b...,1574156421,2019-11-19 13:40:21,2019-11-19,1,0,0,0,0
6,0xffa34c81051336caed93ab7d8b5a54aa954cde213266...,1574157302,2019-11-19 13:55:02,2019-11-19,1,1,1,0,0
7,0x103c1387767a9b8be9c3c0b14915ef48586ac94fab80...,1574157335,2019-11-19 13:55:35,2019-11-19,1,0,0,0,0
8,0x9cbe8c8aeac3552126dd9d52fe7d88d7825ce46ef5c5...,1574157447,2019-11-19 13:57:27,2019-11-19,1,1,1,0,0
9,0x54e5805031f9a209c31f4c013b09a4a80e174bb21d8c...,1574157769,2019-11-19 14:02:49,2019-11-19,1,1,1,0,0


In [9]:
if len(set(ds_proxy_summary["hash"].values) - set(data["hash"].values)) == 0:
    print(f"All DSProxy transaction hashes included in ADDRESS_OF_INTEREST transactions")

All DSProxy transaction hashes included in ADDRESS_OF_INTEREST transactions


In [10]:
def token_txs_by_tx_hash(eth_data, hash):
    return [a for a in eth_data.token_txs if a["hash"] == hash]

def internal_txs_by_tx_hash(eth_data, hash):
    return [a for a in eth_data.internal_txs if a["hash"] == hash]

def make_token_message(token_tx):
    value = float(token_tx["value"]) / 10 ** int(token_tx["tokenDecimal"])
    token_name = token_tx["tokenName"]
    token_from = get_address_name(token_tx["from"])
    token_to = get_address_name(token_tx["to"])
    if token_from == "ZERO Address":
        return f"{value:.3f} {token_name} minted to {token_to}"
    return f"{value:.3f} {token_name} transferred from {token_from} to {token_to}"

In [11]:
def analyze_tx(n, row):
    hash = row["hash"]
    print(f"{n} - {row['datetime']} {hash}")
    if row["normal"] < 1:
        print("External transaction")
        token_txs = token_txs_by_tx_hash(address_etherscan_data, row["hash"])
        for token_tx in token_txs:
            token_msg = make_token_message(token_tx)
            print(token_msg)
        return
    tx = [a for a in address_etherscan_data.txs if a["hash"] == row["hash"]][0]
    # Eth inflow
    if tx["input"] == "0x":
        value = float(tx["value"]) / 10 ** 18
        if get_address_name(tx["from"]) == "ADDRESS_OF_INTEREST":
            print(f"OUTFLOW ETH {value} to {get_address_name(tx['to'])}")
        elif get_address_name(tx["to"]) == "ADDRESS_OF_INTEREST":
            print(f"INFLOW ETH {value} from {get_address_name(tx['from'])}")
        else:
            print("Eth movement not involving ADDRESS_OF_INTEREST")
        return
    if get_address_name(tx["from"]) == "ADDRESS_OF_INTEREST":
        if tx["to"] not in address_of_interest_contracts:
            print(f"Unknown contract: {tx['to']}")
            print(tx)
            return
        msg = address_of_interest_contracts[tx["to"]](row, tx)
        if msg is None:
            print(tx)
            print("HERE NONE")
        print(msg)

def analyze_session(n: int):
    session = data[data["session"] == n]
    for i, row in session.iterrows():
        analyze_tx(i, row)
        print("")

In [12]:
known_contracts = {
    "0x4678f0a6958e4d2bc4f1baf7bc52e8f3564f3fe4",
    "0xd848f54280f8fe8661b796e3bb8d8922c87af452",
    "0x6b175474e89094c44da98b954eedeac495271d0f",
    "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    "0x39755357759ce0d7f32dc8dc45414cca409ae24e",
    "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
    "0xc73e0383f3aff3215e6f04b0331d58cecf0ab849",
    "0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359",
    "0x794e6e91555438afc3ccf1c5076a74f42133d08d",
    "0x868ba9aeaca5b73c7c27f3b01588bf4f1339f2bc",
    "0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2",
    "0xcfeed3fbefe9eb09b37539eaa0ddd58d1e1044ca",
    "0xf9be8f0945acddeedaa64dfca5fe9629d0cf8e5d",
    "0xe41d2489571d322189246dafa5ebde1f4699f498",
    "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",
    "0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407",
    "0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b",
    "0xc00e94cb662c3520282e6f5717214004a7f26888",
    "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",
    "0xdac17f958d2ee523a2206206994597c13d831ec7",
    "0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9",
    "0x960b236a07cf122663c4303350609a66a7b288c0",
    "0x408e41876cccdc0f92210600ef50372656052a38",
    "0xd26114cd6ee289accf82350c8d8487fedb8a0c07",
    "0xd3a9fe267852281a1e6307a1c37cdfd76d39b133",
    "0xa74476443119a942de498590fe1f2454d7d4ac0d",
    "0x221657776846890989a759ba2973e427dff5c9bb",
    "0x0a3f6849f78076aefadf113f5bed87720274ddc0",
    "0x7a250d5630b4cf539739df2c5dacb4c659f2488d",
    "0x1985365e9f78359a9b6ad760e32412f4a445e862",
    "0xa618e54de493ec29432ebd2ca7f14efbf6ac17f7",
    "0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7",
    "0x39aa39c021dfbae8fac545936693ac917d5e7563",
    "0x45f783cce6b7ff23b2ab2d70e416cdb7d6055f51",
    "0x2dded6da1bf5dbdf597c45fcfaa3194e53ecfeaf",
    "0xe592427a0aece92de3edee1f18e0157c05861564",
    "0x0000000000085d4780b73119b644ae5ecd22b376",
    "0x12392f67bdf24fae0af363c24ac620a2f67dad86",
    "0x7d2768de32b0b80b7a3454c06bdac94a69ddc7a9",
    "0xd784927ff2f95ba542bfc824c8a8a98f3495f6b5",
    "0x4da27a545c0c5b758a6ba100e3a049001de870f5",
    "0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9",
    "0xecd5e75afb02efa118af914515d6521aabd189f1",
    "0x890f4e345b1daed0367a877a1612f86a1f86985f",
    "0xa47c8bf37f92abed4a126bda807a7b7498661acd",
    "0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45",
    "0x1111111254fb6c44bac0bed2854e76f90643097d",
    "0x55b916ce078ea594c10a874ba67ecc3d62e29822",
    "0x2260fac5e5542a773aa44fbcfedf7c193bc2c599",
    "0x1111111254eeb25477b68fb85ed929f73a960582",
    "0xae7ab96520de3a18e5e111b5eaab095312d7fe84",
    "0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0",
    "0x99a58482bd75cbab83b27ec03ca68ff489b5788f",
    "0xa6d19194da3b357a122fc7e35f7ce9ec00dbb496",
    "0xdd3f50f8a6cafbe9b31a427582963f465e745af8"
}

## Parse transactions

In [13]:
explorer = TransactionExplorer(address_etherscan_data, ds_proxy_data, addresses_db, known_contracts)

In [14]:
# Load ETH prices

eth_price = []
for fname in ["ETH_2019.csv", "ETH_2020_h1.csv", "ETH_2020_h2.csv", "ETH_2021.csv", "ETH_2022.csv", "ETH_2023.csv"]:
    current_data = pd.read_csv(Path("data") / fname, delimiter=";", parse_dates=['timestamp'])
    current_data['date'] = current_data['timestamp'].dt.date
    current_data["indicative_price"] = (current_data["open"] + current_data["close"] + current_data["low"] + current_data["high"]) / 4
    eth_price.append(current_data)
eth_price = pd.concat(eth_price)

In [15]:
transaction_data = []
for i, row in data.iterrows():
    tx_data = explorer.process_transaction(row, i)
    transaction_data.append(tx_data)

## Print assets movements with balances after each transaction

In [16]:
from collections import defaultdict

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

vaults = {"ADDRESS_OF_INTEREST": defaultdict(int),
          "Related Party": defaultdict(int),
          "DSProxy": defaultdict(int),
          "Eth2Dai: Old Contract": defaultdict(int),
          "Maker: MCD Join ETH A": defaultdict(int),
          "ZERO Address": defaultdict(int)}

assets_of_interest = ["ETH", "WETH", "DAI", "SAI", "USDT", "TUSD", "USDC"]

def format_vaults(vaults: dict, assets_of_interest: list = assets_of_interest) -> pd.DataFrame:
    def format_with_commas(x):
        return f"{x:,.2f}".replace(",", " ").replace(".", ",")
    data = pd.DataFrame.from_records(vaults).fillna(0)
    data = data.applymap(format_with_commas)
    return data

def update_vaults(transfer, vaults):
    if transfer.name not in assets_of_interest:
        return
    if transfer.from_address in vaults:
        vaults[transfer.from_address][transfer.name] -= transfer.value
    if transfer.to_address in vaults:
        vaults[transfer.to_address][transfer.name] += transfer.value

def get_token_value(token: str, value: float, t_date) -> float:
    match token:
        case "DAI" | "USDT" | "USDC":
            return value
        case "ETH" | "WETH":
            cur_ETH_price = eth_price[eth_price["date"] == t_date]["indicative_price"].values[0]
            return transfer.value * cur_ETH_price
        case _:
            return 0

def get_token_value_str(token: str, value: float, t_date):
    match token:
        case "DAI" | "USDT" | "USDC":
            value = f"{value:,.2f}"
        case "ETH" | "WETH":
            cur_ETH_price = eth_price[eth_price["date"] == t_date]["indicative_price"].values[0]
            value = f"{transfer.value * cur_ETH_price:,.2f}"
        case _:
            value = "UNKNOWN"
    return value

for i, tx_data in enumerate(transaction_data):
    if tx_data.error:
        print(f"{i}) {tx_data.datetime} {tx_data.hash} - transaction error")
        continue
    if tx_data.external:
        print(f"{i}) {tx_data.datetime} {tx_data.hash} - EXTERNAL")
    elif tx_data.contract:
        print(f"{i}) {tx_data.datetime} {tx_data.hash} - {tx_data.contract} - {tx_data.function}")
    else:
        print(f"{i}) {tx_data.datetime} {tx_data.hash} - ETH transfer")
    for transfer in tx_data.eth_movements:
        update_vaults(transfer, vaults)
        cur_ETH_price = eth_price[eth_price["date"] == tx_data.datetime.date()]["indicative_price"].values[0]
        if transfer.from_address == "ADDRESS_OF_INTEREST":
            print(f"Sent ETH to {transfer.to_address} -{transfer.value:,.2f} (~${transfer.value * cur_ETH_price:,.2f})".replace(",", " ").replace(".", ","))
            continue
        elif transfer.to_address == "ADDRESS_OF_INTEREST":
            print(f"Received ETH from {transfer.from_address} {transfer.value:,.2f} (~${transfer.value * cur_ETH_price:,.2f})".replace(",", " ").replace(".", ","))
            continue
        else:
            print(f"---- Move {transfer.value:,.2f} ETH (~${transfer.value * cur_ETH_price:,.2f}) from {transfer.from_address} to {transfer.to_address}".replace(",", " ").replace(".", ","))

    for transfer in tx_data.tokens_movements:
        update_vaults(transfer, vaults)
        usd_value = get_token_value_str(transfer.name, transfer.value, tx_data.datetime.date())
        if transfer.from_address == "ADDRESS_OF_INTEREST":
            print(f"Sent {transfer.name} to {transfer.to_address} -{transfer.value:,.2f} (~${usd_value})".replace(",", " ").replace(".", ","))
            continue
        elif transfer.to_address == "ADDRESS_OF_INTEREST":
            print(f"Received {transfer.name} from {transfer.from_address} {transfer.value:,.2f} (~${usd_value})".replace(",", " ").replace(".", ","))
            continue
        else:
            print(f"---- Move {transfer.value:,.2f} (~${usd_value}) {transfer.name} from {transfer.from_address} to {transfer.to_address}".replace(",", " ").replace(".", ","))
    
    if len(tx_data.eth_movements) + len(tx_data.tokens_movements) > 0:
        print("\nBalances:")
        print(format_vaults(vaults))
    else:
        print("No movements")
    print("")

0) 2019-11-19 10:49:41 0x1956d7a97e04141c75af0900e30e9fa0a813081141fbeac74991554baa5c8cbc - ETH transfer
Received ETH from Related Party 1,00 (~$178,92)

Balances:
    ADDRESS_OF_INTEREST DSProxy Eth2Dai: Old Contract Maker: MCD Join ETH A Related Party ZERO Address
ETH                1,00    0,00                  0,00                  0,00         -1,00         0,00

1) 2019-11-19 10:51:02 0x63128cd3e4d74c6f08db5687a49863eb2aa83d2416f6d3ea999dc0100668cecf - ETH transfer
Received ETH from Related Party 1 000,00 (~$178 920,78)

Balances:
    ADDRESS_OF_INTEREST DSProxy Eth2Dai: Old Contract Maker: MCD Join ETH A Related Party ZERO Address
ETH            1 001,00    0,00                  0,00                  0,00     -1 001,00         0,00

2) 2019-11-19 10:59:40 0x8cbd80175791f801a14076683f12c4cc9f2eef790a147e433d1cd83d9b572a9e - Maker: Proxy Registry - 
No movements

3) 2019-11-19 11:04:39 0x4d4c3827b2d91039f326ec6205a7f17067e378bb3319f3dd88ba906203a00fb5 - DSProxy - execute
Sent ETH 

## Save trasactions ledger to csv for further analysis

In [17]:
# make transaction ledger in csv for further analysis

# N | hash | datetime | from | to | asset | value | usd | description

with open("tx_ledger.csv", "w") as f:
    f.write(f"#, datetime, from, to, asset, value, usd, description, hash\n")
    for n, row in data.iterrows():
        tx_data = explorer.process_transaction(row, n)
        if tx_data.error:
            continue
        if tx_data.external:
            description = "EXTERNAL"
        elif tx_data.contract:
            description = f"{tx_data.contract} - {tx_data.function}"
        else:
            description = "ETH transfer"
        if len(tx_data.eth_movements + tx_data.tokens_movements) == 0:
            continue
        for transfer in tx_data.eth_movements + tx_data.tokens_movements:
            from_addr = transfer.from_address
            to_addr = transfer.to_address
            value = transfer.value
            usd_value = get_token_value(transfer.name, transfer.value, tx_data.datetime.date())
            asset = transfer.name
            if from_addr != "ADDRESS_OF_INTEREST" and to_addr != "ADDRESS_OF_INTEREST":
                continue
            f.write(f"{n}, {row['datetime']}, {from_addr}, {to_addr}, {asset}, {value}, {usd_value}, {description}, {row['hash']}\n")